In [ ]:
import pandas as pnd
import numpy as np
pnd.option_context('display.max_rows', 10)


In [ ]:
#Chamando os dados do mes
dados = pnd.read_csv('month_2.csv').sort_values(by='datetime')
dados


In [ ]:
dados.describe()
#Não é uma duvida de código, é mais tipo QUE CARALHOS DE NUMERO
#o engraçado é que fazzendo o boxplot TUDO é outlier
#nao deveria ser
#Só achei curioso

In [ ]:
#Divide tudo em classes, não retorna nada porque não pedimos para retornar
classes = [0, 2000, 10000, 100000, 500000]
labels = ['Muito Pouco', 'Pouco', 'Médio', 'Muito']
porcentagemValores = pnd.value_counts(pnd.cut(x = dados.pulseCount, bins = classes, labels=labels, include_lowest=True), normalize = True) * 100
quantidadeValores = pnd.value_counts(pnd.cut(x=dados.pulseCount, bins = classes, labels=labels, include_lowest=True))
dataFreqPor =pnd.DataFrame({'Frequencia': quantidadeValores, 'Porcentagem': porcentagemValores})


In [ ]:
#Usando a formula de Sturges para fazer a separação
n = dados.shape[0]
k = 1+(10/3) * np.log10(n)
k = int(k.round(0))
print(dados.pulseCount.min())
dataSturges = pnd.value_counts(pnd.cut(x=dados.pulseCount, bins=20, include_lowest=True), normalize=True, sort=False)*100
dataSturges

In [ ]:
#Aqui claramente percebemos que não funciona sem limpar os dados
import seaborn as sb
dados.pulseCount.hist(bins=20, figsize=(12,6))
ax = sb.histplot(dados.pulseCount, kde=True)
ax.figure.set_size_inches(12,6)
ax.set_title('Distribuição da frequência de pulsos', fontsize=18)
ax.set_xlabel('Consumo', fontsize=10)

In [ ]:
frequenciaGasto = pnd.value_counts(pnd.cut(x=dados.meterIndex, bins=20, include_lowest=True))
graficoFrequenciaGasto = pnd.DataFrame({'Frequencia': frequenciaGasto})
graficoFrequenciaGasto['Frequencia'].plot.bar(width=1, color='blue', figsize=(12,6))
dados.pulseCount.describe()



In [ ]:
#Gráfico limitando a contagem. Ainda sem a correção
ax = sb.histplot(dados.query('pulseCount < 100000').pulseCount, bins=20)
ax.figure.set_size_inches(12,6)
ax

In [ ]:
#Moda, média e mediana. Vale a pena tentar com os dados corrigidos
moda = dados.pulseCount.mode()
mediana = dados.pulseCount.median()
media = dados.pulseCount.mean()
print('Média: %f, Mediana: %f, Moda: %f' % (media, mediana, moda))

In [ ]:
#Tentativa extremamente falha de fazer um boxplot. Como eu era ingênuo achando que ia ser fácil
ax = sb.boxplot(x= 'pulseCount', data=dados, orient='h')
ax.figure.set_size_inches(12,6)
ax.set_title('Boxplot da frequência de pulsos', fontsize=18)
ax.set_xlabel('Consumo', fontsize=14)
ax

In [ ]:
#Tentativa de dividir os dados em decis. Vale a pena avaliar depois de consertar a desgraça que está depois no código
print(dados.meterIndex.quantile([i / 10 for i in range(1, 10)])) #Decis
print(dados.meterIndex.max()) #Alguma coisa parece severamente errada com o valor máximo
moda = dados.meterIndex.mode()
mediana = dados.meterIndex.median()
media = dados.meterIndex.mean()#Nédia também está muito estranha, esses dados estão ao longo do grafico inteiro
print('Média: %f, Mediana: %f, Moda: %f' % (media, mediana, moda))

In [ ]:
#Esse é o famoso cliente que existe no medidor e não existe nas informacoes cadastrais. Isso me fez dar uma volta que só Deus sabe a lógica que implementei
maxValueIndex = dados['pulseCount'].idxmax()
maxValueRow = dados.loc[maxValueIndex]
print(maxValueRow)

In [ ]:
#Cria uma função extendida de descrever
def describeExtended(data):
    description = data.describe()
    description.loc['var'] = data.var()
    description.loc['median'] = data.median()
    description.loc['mode'] = data.mode().iloc[0]
    return description

In [ ]:
#PulseCount
print(describeExtended(dados.pulseCount))

In [ ]:
#MeterIndex
print(describeExtended(dados.meterIndex))

In [ ]:
#InitialIndex
print(describeExtended(dados.initialIndex))

In [ ]:
#Chama a tabela de informação cadastral e verifica quantos tipos de situação tem; São 4 se está curioso
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')
dadosCadastrais['situacao'].unique()

In [ ]:
#O nome é bem explicativo, foi minha verificação que cada sensor poderia ter mais de um cliente
sensorBabaca = dados[dados['meterSN'].str.contains('002A93068D6C')]
sensorBabaca

In [ ]:
aaaaaaaaaaaaaaaaa = dados[dados['meterSN'].str.contains('E0D5407L3025')].sort_values(by='datetime')
aaaaaaaaaaaaaaaaa
dados['inputType'].unique()


In [ ]:
#Nesse momento o código está calculando a base do pulseCount com referência ao contador
#O problema é que tem contadores com mais de um cliente, mais de um tipo de medidor e medidas COMPLETAMENTE DIFERENTES
#Focar no código do cliente ao invés do meterSN é inútil da mesma maneira, um cliente tem mais de um medidor
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique()
mesFiltrado = dados[dados['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime')
mesFiltrado = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']
resultado = mesFiltrado.groupby(['meterSN', 'clientCode']).first()
resultado = resultado[['pulseCount']]
final = pnd.DataFrame({'pulseCount2': resultado.pulseCount})
merged_df = pnd.merge(dados, final, on=['meterSN', 'clientCode'], how='left')
merged_df['pulseCount'] = merged_df['pulseCount'] - merged_df['pulseCount2']
merged_df.describe()

In [ ]:
merged_df.loc[merged_df.pulseCount.idxmin()]

In [ ]:
dadosComparacao = merged_df[merged_df['clientCode'].str.contains('4b2b41142406be0d6b8c4f660d757b8a5c8c3307395eca') | merged_df['clientCode'].str.contains('cd52774601bea7ac814b4bbb41be1b9f3eae936dd63128')]
sb.lineplot(x='datetime', y='pulseCount', hue='clientCode', data=dadosComparacao)

In [ ]:
merged_df.to_csv('month_2_clean.csv', index=False)

In [ ]:
#Esse gráfico ainda não tá funcionando
print(merged_df.pulseCount.max)
sb.displot(data=merged_df, x='pulseCount', fill=True)

In [ ]:
#Lembrar de arrumar
#ultimoDoMes = mesFiltrado.groupby(['meterSN', 'clientCode']).last()
#sb.histplot( x='pulseCount', hue='clientCode', data=ultimoDoMes, legend=False)